In [3]:
from math import *
import numpy as np
np.random.seed(0)

In [10]:
def generarTiempo(t, l): 
    t = t - (np.log(np.random.uniform())/l)
    return t

def simulation(serverNumber, request, LAMBDA):
    T = 3600
    t = Na = Nd = 0 #t es por donde vamos
    D = []
    A = []
    td = [] #departure
    ta = []
    n = []
    pri = False
    def smaller(arrayValues, arrayCounter ):
        pos, value = 0, inf
        for i in range(serverNumber):
            if (arrayValues[i] < value and arrayCounter[i] != 0):
                value = arrayValues[i]
                pos = i
        return pos

    for i in range(serverNumber):
        D.append([])
        A.append([])
        n.append(0)
        ta.append(0)
        td.append(inf)
    flag = True
    while (flag):
        posTa = ta.index(min(ta))
        posTd = td.index(min(td))
        if (ta[posTa] <= td[posTd] and ta[posTa] <= T): # La siguiente operacion que atendemos es una llegada
            #Move pointer
            t = ta[ta.index(max(ta))] #Último tiempo de llegada
            #Counters
            Na+=1
            n[posTa]+= 1
            #Generate next
            ta[posTa] = generarTiempo(t, LAMBDA)
            if (n[posTa] == 1):
                td[posTa] = t - (1/request)*np.log(np.random.uniform())
            A[posTa].append(t) #Tiempo de llegada de la solicitud
        elif (td[posTd] <= ta[posTa] and td[posTd] <= T): #Attending exit case
            #Move pointer
            t = td[posTd]
            #Counters
            Nd+=1
            n[posTd]-=1
            #Last element in list
            if (n[posTd] == 0):
                td[posTd] = inf
            else:
                td[posTd] = t - (1/request)*np.log(np.random.uniform())
            #print(t, td, ta)
            D[posTd].append(t) #Tiempo de salida de la solicitud
        elif (min(ta[posTa],td[posTd]) > T and sum(n) > 0):
            pos = smaller(td, n)
            pri = True
            t = td[pos]
            Nd+=1
            n[pos]-=1
            if (n[pos] > 0):
                td[pos] = t - (1/request)*np.log(np.random.uniform())
            D[pos].append(t)#Tiempo de salida de la solicitud
        elif (min(ta[posTa], td[posTd]) > T and sum(n) == 0): 
            Tp = max(t - T, 0)
            flag = False
    return A, D

def info(listArrival,listDeparture,val=True):
    canti=len(listArrival)
    ocup=0
    for i in range(canti):
        ocup=ocup+listDeparture[i]-listArrival[i]
    us=listDeparture[(canti-1)]
    libre=0
    cola=0
    conta=0
    for j in range(canti-1):
        if (listDeparture[j]>listArrival[j+1]):
            dif=listDeparture[j]-listArrival[j+1]
            cola=cola+dif
            conta+=1
        elif (listDeparture[j]<listArrival[j+1]):
            dif=listArrival[j+1]-listDeparture[j]
            libre=libre+dif
    espprom=cola/canti 
    ocup=ocup-cola
    tiempotot=listDeparture[canti-1]-listArrival[0]
    colasec=conta/tiempotot
    return canti,ocup,libre,cola,espprom,colasec,us


def cantidadNecesaria(LAMBDA):  
    flag=True
    cont=1
    while(flag==True):
        A,D = simulation(cont,10,LAMBDA)
        for i in range(cont):
            canti,ocup,libre,cola,espprom,colasec,us=info(A[i],D[i],False)
            #print(espprom)
            if(round(espprom,10)==0):
                flag=False
                cantidad=len(A)
            else:
                cont+=1
    print("Para manejar ",str(LAMBDA*60)," solicitudes por minuto y que siempre haya un servidor de Ant Smart Computing disponible, se necesitan alquilar ", cantidad, " servidores")



In [11]:
#Task 1   
ServerAmount= 10
LAMBDA=40
A,D = simulation(ServerAmount,10,LAMBDA)
Ag,Dg= simulation(1,100,LAMBDA)

canti,ocup,libre,cola,espprom,colasec,us=info(Ag[0],Dg[0])
print("El servidor de Gorilla Megacomputing atendio ",canti, " solicitudes, paso ocupado ",ocup," segundos y desocupado ",libre," segundos. \n Las solicitudes estuvieron en total ",cola, " segundos en cola, lo cual nos da un promedio de ",espprom, " segundos por solicitud. \n Cada segundo en promedio hubo ",colasec," solicitudes en cola.\n El momento de salida de la ultima solicitud fue en el segundo: ",us)   
print("---------------------------------------------------------------------------------------------------------------------------")
salida=[]
promcola=0
cantidad=0
for i in range(ServerAmount):
    canti,ocup,libre,cola,espprom,colasec,us=info(A[i],D[i])
    promcola=promcola+cola
    cantidad+=canti
    salida.append(us)
    print("El servidor ", i+1, " de Ants Smart Computing atendio ",canti, " solicitudes, paso ocupado ",ocup," segundos y desocupado ",libre," segundos. \n Las solicitudes estuvieron en total ",cola, " segundos en cola, lo cual nos da un promedio de ",espprom, " segundos por solicitud. \n Cada segundo en promedio hubo ",colasec," solicitudes en cola.\n El momento de salida de la ultima solicitud fue en el segundo: ",us,"\n")   
prom=promcola/cantidad
print("---------------------------------------------------------------------------------------------------------------------------")
print("Resumen para los",i+1, " servidores: \n Se atendieron ", cantidad," solicitudes. La ultima salida fue en el  segundo: ",max(salida), ". El tiempo promedio de espera de cada solicitud fue de: ",prom," segundos.")

El servidor de Gorilla Megacomputing atendio  143052  solicitudes, paso ocupado  1434.8389108867211  segundos y desocupado  2165.199114215297  segundos. 
 Las solicitudes estuvieron en total  949.7633659799023  segundos en cola, lo cual nos da un promedio de  0.006639287573608914  segundos por solicitud. 
 Cada segundo en promedio hubo  15.857610281319591  solicitudes en cola.
 El momento de salida de la ultima solicitud fue en el segundo:  3600.038025102066
---------------------------------------------------------------------------------------------------------------------------
El servidor  1  de Ants Smart Computing atendio  14391  solicitudes, paso ocupado  413.327762243709  segundos y desocupado  3186.8907504196936  segundos. 
 Las solicitudes estuvieron en total  169.141941411282  segundos en cola, lo cual nos da un promedio de  0.011753313974795497  segundos por solicitud. 
 Cada segundo en promedio hubo  0.38442111086644365  solicitudes en cola.
 El momento de salida de la ulti

In [4]:
#Task 2
cantidadNecesaria(LAMBDA)

Para manejar  2400  solicitudes por minuto y que siempre haya un servidor de Ant Smart Computing disponible, se necesitan alquilar  32  servidores


In [12]:
#Task 3   
ServerAmount= 10
LAMBDA=100
A,D = simulation(ServerAmount,10,LAMBDA)
Ag,Dg= simulation(1,100,LAMBDA)

canti,ocup,libre,cola,espprom,colasec,us=info(Ag[0],Dg[0])
print("El servidor de Gorilla Megacomputing atendio ",canti, " solicitudes, paso ocupado ",ocup," segundos y desocupado ",libre," segundos. \n Las solicitudes estuvieron en total ",cola, " segundos en cola, lo cual nos da un promedio de ",espprom, " segundos por solicitud. \n Cada segundo en promedio hubo ",colasec," solicitudes en cola.\n El momento de salida de la ultima solicitud fue en el segundo: ",us)   
print("------------------------------------------------------------------------------------------------------------------------")
salida=[]
promcola=0
cantidad=0
for i in range(ServerAmount):
    canti,ocup,libre,cola,espprom,colasec,us=info(A[i],D[i])
    promcola=promcola+cola
    cantidad+=canti
    salida.append(us)
    print("El servidor ", i+1, " de Ants Smart Computing atendio ",canti, " solicitudes, paso ocupado ",ocup," segundos y desocupado ",libre," segundos. \n Las solicitudes estuvieron en total ",cola, " segundos en cola, lo cual nos da un promedio de ",espprom, " segundos por solicitud. \n Cada segundo en promedio hubo ",colasec," solicitudes en cola.\n El momento de salida de la ultima solicitud fue en el segundo: ",us,"\n")   
prom=promcola/cantidad
print("---------------------------------------------------------------------------------------------------------------------------")
print("Resumen para los",i+1, " servidores: \n Se atendieron ", cantidad," solicitudes. La ultima salida fue en el  segundo: ",max(salida), ". El tiempo promedio de espera de cada solicitud fue de: ",prom," segundos.")

El servidor de Gorilla Megacomputing atendio  360687  solicitudes, paso ocupado  3610.073354749009  segundos y desocupado  1.5261548558612448  segundos. 
 Las solicitudes estuvieron en total  3724011.8336244724  segundos en cola, lo cual nos da un promedio de  10.32477420484928  segundos por solicitud. 
 Cada segundo en promedio hubo  99.8294520264208  solicitudes en cola.
 El momento de salida de la ultima solicitud fue en el segundo:  3611.5995097777222
------------------------------------------------------------------------------------------------------------------------
El servidor  1  de Ants Smart Computing atendio  36116  solicitudes, paso ocupado  3625.5186776468763  segundos y desocupado  11.179966775942296  segundos. 
 Las solicitudes estuvieron en total  817720.6700218446  segundos en cola, lo cual nos da un promedio de  22.641507088875972  segundos por solicitud. 
 Cada segundo en promedio hubo  9.891113759218426  solicitudes en cola.
 El momento de salida de la ultima soli

In [6]:
#Task 4
cantidadNecesaria(LAMBDA)

Para manejar  6000  solicitudes por minuto y que siempre haya un servidor de Ant Smart Computing disponible, se necesitan alquilar  128  servidores
